In [2]:
import json

f = open('DFA.json')
DFA = json.load(f)
f.close()

states = DFA.keys()

for state in states:
    print(state , DFA.get(state) , sep =" : ")

StartingState : A
A : {'states': ['S0', 'S2', 'S7', 'S4', 'S1'], 'IsTerminating': False, 'a': 'B', 'b': 'C'}
B : {'states': ['S2', 'S7', 'S4', 'S8', 'S6', 'S3', 'S1'], 'IsTerminating': False, 'a': 'B', 'b': 'D'}
C : {'states': ['S2', 'S5', 'S7', 'S4', 'S6', 'S1'], 'IsTerminating': False, 'a': 'B', 'b': 'C'}
D : {'states': ['S2', 'S5', 'S9', 'S7', 'S4', 'S6', 'S1'], 'IsTerminating': False, 'a': 'B', 'b': 'E'}
E : {'states': ['S10', 'S2', 'S5', 'S7', 'S4', 'S6', 'S1'], 'IsTerminating': True, 'a': 'B', 'b': 'C'}


In [3]:
def get_transitions(DFA):
    transitions = []
    for state in DFA.keys():
        for transition in DFA.get(state):
            if transition not in transitions and transition != 'states' and transition != 'IsTerminating' and state != 'StartingState' : 
                transitions.append(transition)
    return transitions

def get_accepting_states(DFA):
    accepting_states = []
    for state in DFA.keys():
        if state != "StartingState" and DFA.get(state).get('IsTerminating') == True:
            accepting_states.append(state)
    return accepting_states

def get_non_accepting_states(DFA):
    non_accepting_states = []
    for state in DFA.keys():
        if state != "StartingState" and DFA.get(state).get('IsTerminating') == False:
            non_accepting_states.append(state)
    return non_accepting_states

In [4]:
transitions = get_transitions(DFA)
print(transitions)

accepting_states = get_accepting_states(DFA)
print(accepting_states)

non_accepting_states = get_non_accepting_states(DFA)
print(non_accepting_states)

['a', 'b']
['E']
['A', 'B', 'C', 'D']


In [5]:
def Minimum_DFA(DFA):
    partitions = [get_accepting_states(DFA) , get_non_accepting_states(DFA)]
    transitions = get_transitions(DFA)
    for transition in transitions: # for each transition in the DFA e.g. a , b
        new_partitions = [] # new partitions after the transition
        for partition in partitions: # for each partition in the partitions e.g. [A,B,C] , [D] 
            partition_dict = {}
            for state in partition: # for each state in the partition e.g. A , B , C
                if DFA.get(state).get(transition) in partition_dict.keys(): # if the transition of the state is already in the partition_dict ,transition is The state that the transition leads to
                    partition_dict.get(DFA.get(state).get(transition)).append(state) 
                else:
                    partition_dict[DFA.get(state).get(transition)] = [state]
            for key in partition_dict.keys(): # for each key in the partition_dict e.g. A , B , C
                new_partitions.append(partition_dict.get(key)) # add the partition to the new_partitions
        partitions = new_partitions # update the partitions
    return partitions

Minimum_DFA(DFA)

[['E'], ['A', 'C'], ['B'], ['D']]

In [6]:
# merge the partitions to get the minimum DFA
def merge_partitions(DFA,partitions):
    for partition in partitions:
        if len(partition) > 1:
            for state in partition:
                if state != partition[0]:
                    DFA.get(partition[0]).update(DFA.get(state))
                    del DFA[state]
                
                for state in DFA.keys():
                    for transition in DFA.get(state):
                        if transition != 'states' and transition != 'IsTerminating' and state != 'StartingState':
                            if DFA.get(state).get(transition) in partition:
                                DFA.get(state)[transition] = partition[0]
                
    return DFA

merge_partitions(DFA,Minimum_DFA(DFA))

{'StartingState': 'A',
 'A': {'states': ['S2', 'S5', 'S7', 'S4', 'S6', 'S1'],
  'IsTerminating': False,
  'a': 'B',
  'b': 'A'},
 'B': {'states': ['S2', 'S7', 'S4', 'S8', 'S6', 'S3', 'S1'],
  'IsTerminating': False,
  'a': 'B',
  'b': 'D'},
 'D': {'states': ['S2', 'S5', 'S9', 'S7', 'S4', 'S6', 'S1'],
  'IsTerminating': False,
  'a': 'B',
  'b': 'E'},
 'E': {'states': ['S10', 'S2', 'S5', 'S7', 'S4', 'S6', 'S1'],
  'IsTerminating': True,
  'a': 'B',
  'b': 'A'}}

In [7]:
f = open('Min_DFA.json','w')
json.dump(DFA,f)
f.close()
